<a href="https://colab.research.google.com/github/saadatialirezam-create/glaucoma-detection/blob/main/04_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from google.colab import drive
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

## Mount Drive

In [5]:
drive.mount('/content/drive')
PRED_DIR1 = "/content/drive/MyDrive/Glaucoma_data/preds"
PRED_DIR2 = "/content/drive/MyDrive/Glaucoma_data_preds"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load Predictions

In [9]:
y_true   = np.load(f"{PRED_DIR1}/y_true.npy")

res_pred = np.load(f"{PRED_DIR1}/resnet50_pred.npy")
vgg_pred = np.load(f"{PRED_DIR2}/vgg16_pred.npy")
rf_pred  = np.load(f"{PRED_DIR1}/rf_pred.npy")

print("Shapes:")
print("y_true:", y_true.shape)
print("ResNet:", res_pred.shape)
print("VGG   :", vgg_pred.shape)
print("RF    :", rf_pred.shape)

Shapes:
y_true: (475,)
ResNet: (475,)
VGG   : (475,)
RF    : (475,)


## Majority Voting (2 out of 3)

In [13]:
votes = res_pred + vgg_pred + rf_pred
ensemble_pred_hard = (votes >= 2).astype(np.int64)

## Metrics

In [14]:
acc = accuracy_score(y_true, ensemble_pred_hard)
prec = precision_score(y_true, ensemble_pred_hard, zero_division=0)
rec = recall_score(y_true, ensemble_pred_hard, zero_division=0)
f1 = f1_score(y_true, ensemble_pred_hard, zero_division=0)
cm = confusion_matrix(y_true, ensemble_pred_hard)

print("\n--- Ensemble (Majority Voting) ---")
print("Accuracy :", acc)
print("Precision:", prec)
print("Recall   :", rec)
print("F1-score :", f1)
print("Confusion Matrix:\n", cm)


--- Ensemble (Majority Voting) ---
Accuracy : 0.7789473684210526
Precision: 0.875
Recall   : 0.4745762711864407
F1-score : 0.6153846153846154
Confusion Matrix:
 [[286  12]
 [ 93  84]]


## Soft Voting

In [15]:
res_prob = np.load(f"{PRED_DIR1}/resnet50_prob.npy")
vgg_prob = np.load(f"{PRED_DIR2}/vgg16_prob.npy")
rf_prob  = np.load(f"{PRED_DIR1}/rf_prob.npy")

avg_prob = (res_prob + vgg_prob + rf_prob) / 3
ensemble_pred_soft = (avg_prob >= 0.5).astype(np.int64)

## Metrics

In [16]:
acc = accuracy_score(y_true, ensemble_pred_soft)
prec = precision_score(y_true, ensemble_pred_soft, zero_division=0)
rec = recall_score(y_true, ensemble_pred_soft, zero_division=0)
f1 = f1_score(y_true, ensemble_pred_soft, zero_division=0)
cm = confusion_matrix(y_true, ensemble_pred_soft)

print("\n--- Ensemble (Majority Voting) ---")
print("Accuracy :", acc)
print("Precision:", prec)
print("Recall   :", rec)
print("F1-score :", f1)
print("Confusion Matrix:\n", cm)


--- Ensemble (Majority Voting) ---
Accuracy : 0.7936842105263158
Precision: 0.9072164948453608
Recall   : 0.4971751412429379
F1-score : 0.6423357664233577
Confusion Matrix:
 [[289   9]
 [ 89  88]]
